In [115]:
import pandas as pd
import numpy as np

In [116]:
df = pd.read_csv("spam.csv",encoding="utf-8")[["v1","v2"]]
df.drop_duplicates(inplace=True)

In [117]:
df = df.reset_index()

### 0 -> spam & 1 -> ham

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[["spam","ham"]]) 
encoded = oe.fit_transform(df[["v1"]])
d2 = pd.DataFrame(encoded,columns=oe.get_feature_names_out())
df["is_spam"] = d2[oe.get_feature_names_out()]

In [119]:
df["is_spam"] = df["is_spam"].astype("int")

In [120]:
df = df.drop(["index","v1"],axis=1)

In [121]:
df["msg_len"] = df["v2"].str.len()

In [122]:
df.groupby("is_spam")["msg_len"].mean()

is_spam
0    137.455657
1     70.457263
Name: msg_len, dtype: float64

In [126]:
df["v2"][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
import re,string
from textblob import TextBlob
from nltk.corpus import stopwords
class TxtPreprocessing:
    def __init__(self,text):
        self.text = text
        self.new_text = None
        self.exclude = string.punctuation
    def remove_url(self):
        pattern = re.compile(r"https?://\S+|www\.\S+")
        return pattern.sub(r"",self.text)
    def remove_puncs(self):
        return self.text.translate(str.maketrans("","",self.exclude))
    def spell_correct(self):
        t_blob = TextBlob(self.text)
        return t_blob.correct().string
    def rem_stop(self):
        new_text = []
        for word in self.text.split():
            if word in stopwords.split("english"):
                new_text.append("")
            else:
                new_text.append(word)
        x =  new_text[:]
        new_text.clear()
        return " ".join(x)

In [128]:
txtpr = TxtPreprocessing('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')

In [129]:
txtpr